# Continuity checks for encoding methods
This notebook maakes some plots that check whether
Discrete Indicator Vector (DIV) and Multi-Point Proximity (MPP) encodings exhibit 

continuity under shae transformations.

In [ ]:
import numpy as np

import shapely
from shapely.ops import nearest_points

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import MPPEncoder
from geo_encodings import DIVEncoder
from geo_encodings import draw_shape

## Define DIV and MPP encoders

In [ ]:
# Use the same region and resolution definitions for both encoders.
x0, y0 = 0, 0
x1, y1 = 100, 100
resolution = 25

# DIV encoder
div_encoder = DIVEncoder([x0, y0, x1, y1], resolution)
print('DIV encoder: %d tiles' % len(div_encoder))

# MPP encoder
mpp_encoder = MPPEncoder([x0, y0, x1, y1], resolution, center=True)
print('MPP encoder: %d reference points' % len(mpp_encoder))


## Move a point across the region and compute its encodings

In [ ]:
div_encodings = []
mpp_encodings = []

# Define a kind of squiggly trajectory along which we will move a point.
n_positions = 30
point_x = np.linspace(10, 90, n_positions)
point_y = np.linspace(13, 85, n_positions) + 20 * np.sin(np.arange(n_positions) / n_positions * 6.28)

# Get encodings for each point along that trajectory.
for point_coords in list(zip(point_x, point_y)):
    p = shapely.Point(point_coords[0], point_coords[1])
    div_encodings.append(div_encoder.encode(p).values())
    mpp_encodings.append(mpp_encoder.encode(p).values())


In [ ]:
fig = make_subplots(1, 3, subplot_titles=['Trajectory', 'DIV Encoding', 'MPP Encoding'])

position_index = np.arange(n_positions)

# Draw the trajectory for the point.
trace = Scatter(x=point_x, y=point_y, mode='markers+lines',
                marker={'color': 'black'}, showlegend=False)
fig.append_trace(trace, 1, 1)
fig['layout']['xaxis1']['title'] = 'x coordinate'
fig['layout']['yaxis1']['title'] = 'y coordinate'


n_elements = len(div_encoder)
for i in range(n_elements):
    element_values = [encoding[i] for encoding in div_encodings]
    trace = Scatter(x=position_index, y=element_values, name='element %d' % i, 
                    mode='markers+lines', showlegend=False)
    fig.append_trace(trace, 1, 2)
fig['layout']['xaxis2']['title'] = 'position along trajectory'
fig['layout']['yaxis2']['title'] = 'encoding elements'

n_elements = len(mpp_encoder)
for i in range(n_elements):
    element_values = [encoding[i] for encoding in mpp_encodings]
    trace = Scatter(x=position_index, y=element_values, name='element %d' % i, 
                    mode='markers+lines', showlegend=False)
    fig.append_trace(trace, 1, 3)
fig['layout']['xaxis3']['title'] = 'position along trajectory'
fig['layout']['yaxis3']['title'] = 'encoding elements'

fig['layout']['width'] = 1000
fig['layout']['height'] = 350
fig.show()


## Nearest point jumps and continuity
If a shape moves by a small amount, it is possible that the closest point to a reference point 
will change. But that will NOT cause a large jump in the value of the encoding, since the
"jump" occurs at a point when the distances to the respective closest points are equal.
Herre I'm making a conmtrioved example to illustrate this point.

In [ ]:
x0, y0, x1, y1 = 0, 0, 100, 100
resolution = 50
encoder = MPPEncoder([x0, y0, x1, y1], resolution, center=True)

In [ ]:
g1 = shapely.wkt.loads('LINESTRING(5 40, 25 10, 45 41, 85 65)')
g2 = shapely.affinity.translate(g1, xoff=0.2)


In [ ]:
e1 = encoder.encode(g1).values()
print('encoding 1:', np.round(e1, 4))

e2 = encoder.encode(g2).values()
print('encoding 2:', np.round(e2, 4))


In [ ]:
def plot1(g, irow, icol):
    trace = Scatter(
        x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
        mode='markers', marker={'color': 'blue', 'size': 8}
    )
    fig.append_trace(trace, irow, icol)

    xx = []
    yy = []
    for rp in encoder.ref_points:
        p = nearest_points(g, rp)[0]
        connector_x = [p.xy[0][0], rp.xy[0][0]]
        connector_y = [p.xy[1][0], rp.xy[1][0]]
        xx = xx + connector_x + [None]
        yy = yy + connector_y + [None]
    trace = Scatter(
        x=xx, y=yy, name='distances',
        mode='lines', line={'dash': 'solid', 'color': 'gray'}
    )
    fig.append_trace(trace, irow, icol)

    draw_shape(g, fig, irow, icol)
    fig['layout']['xaxis%d'%icol]['range'] = [0, 100]
    fig['layout']['yaxis%d'%icol]['range'] = [0, 100]



fig = make_subplots(1, 2, subplot_titles=['original shape', 'shifted by 0.2'])
plot1(g1, 1, 1)
plot1(g2, 1, 2)

fig['layout']['width'] = 790
fig['layout']['height'] = 400

fig.show()